<a href="https://colab.research.google.com/github/hepam4/IntroduccionIA/blob/main/punto_1_open_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print("Versión de openai:", openai.__version__)

Versión de openai: 0.27.8


In [1]:
# Desinstalar cualquier versión anterior de openai e instalar la versión 0.27.8
!pip uninstall openai -y
!pip install openai==0.27.8
!pip install PyPDF2

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Found existing installation: openai 0.27.8
Uninstalling openai-0.27.8:
  Successfully uninstalled openai-0.27.8
  Using cached openai-0.27.8-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.27.8-py3-none-any.whl (73 kB)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import openai
import pandas as pd
from PyPDF2 import PdfReader

Se utiliza otra metodologia para la API de Open AI, debido a que algunos de los componentes beta que se estaban utilizando me arrojaban error.
Si bien el codigo proporcionado en clase utiliza almacenamiento vectorial y su estructura es mas robusta, modular y orientada a objetos,  se busco otra alternativa para pdarle solucion al requerimiento y que a la vez fuera sencilla para lo que se requiere implementar y comprender con el estudio.

In [7]:
# Ruta al archivo con la API key en Drive proporcionada en clase
api_key_path = "/content/drive/MyDrive/Universidad EIA - AI especializacion/Introduccion IA/openapikey.txt"

#Se busco otra alternativa de pasar la api key de manera segura, investigando se sugiere utilizar la variable de entorno por lo que
# Si no existe la variable de entorno OPENAI_API_KEY, la leemos desde el archivo y la asignamos
if "OPENAI_API_KEY" not in os.environ:
    with open(api_key_path, "r") as f:
        os.environ["OPENAI_API_KEY"] = f.read().strip()

# Asignar la API key a openai usando la variable de entorno
openai.api_key = os.environ["OPENAI_API_KEY"]

# Directorio donde se encuentran los documentos de Judo
docs_folder = "/content/drive/MyDrive/Universidad EIA - AI especializacion/Introduccion IA/Judo_Docs"
files = os.listdir(docs_folder)

# Función para leer el contenido de cada archivo según su extensión .txt, .csv o .pdf
documentos = []
for file in files:
    file_path = os.path.join(docs_folder, file)
    ext = os.path.splitext(file)[1].lower()
    content = ""
    if ext == ".txt":
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
    elif ext == ".csv":
        df = pd.read_csv(file_path)
        content = df.to_string(index=False)
    elif ext == ".pdf":
        reader = PdfReader(file_path)
        for page in reader.pages:
            text = page.extract_text()
            if text:
                content += text + "\n"
    else:
        print(f"Formato no soportado: {file}")
        continue
    documentos.append({"titulo": file, "contenido": content})

# Combinar todos los documentos en un unico contexto para el prompt
contexto = "\n\n".join([f"{doc['titulo']}:\n{doc['contenido']}" for doc in documentos]) #aplicamos list comprehension

def obtener_respuesta(pregunta):
    # Construir el prompt combinando el contexto y la pregunta del usuario
    prompt = (
        f"Utilizando la siguiente información sobre Judo:\n\n{contexto}\n\n"
        f"Responde de manera clara y concisa a la siguiente pregunta:\n{pregunta}"
    )

    # Llamada a la API de OpenAI usando el modelo "gpt-4o-mini" sugerido en clase, tambien se probo con "gpt-4"
    respuesta = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": """Eres experto en análisis de judo.
A partir de documentos sobre judo de diferentes temas, debes ser capaz de extraer información de ellos.
Da toda la información sólo de los documentos que tienes.
Si usas tablas, utiliza formato markdown y responde siempre en el idioma que se te pregunte."""},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7, #controlar su creatividad
        max_tokens=200   #controlar length respuestas
    )

    return respuesta.choices[0].message.content.strip()

# Interfaz simple en la consola para interactuar con el sistema experto como usuario
print("Bienvenido al sistema experto en Judo 柔道. Escribe 'salir' para terminar.")
while True: #ciclo hasta que el usuario digite salir
    pregunta = input("Ingresa tu pregunta sobre Judo: \n")
    if pregunta.lower() == "salir":
        break
    respuesta = obtener_respuesta(pregunta)
    print("\n\nRespuesta del asistente en Judo:")
    print(respuesta)
    print("-" * 50) #imprimimos unas lineas para diferenciar las preguntas y respuestas


Bienvenido al sistema experto en Judo 柔道. Escribe 'salir' para terminar.
Ingresa tu pregunta sobre Judo: 
cual es el kanji de judo y que significa


Respuesta del asistente en Judo:
El kanji de judo es 柔道. Este se compone de dos caracteres:

- 柔 (ju) que significa "suave" o "flexible".
- 道 (dō) que significa "camino" o "ruta".

Por lo tanto, "judo" se traduce literalmente como "el camino de la suavidad".
--------------------------------------------------
Ingresa tu pregunta sobre Judo: 
cual es el pais donde mas se practica el judo


Respuesta del asistente en Judo:
El país donde más se practica el judo es Japón, ya que es su país de origen y donde se fundó la disciplina. Sin embargo, a nivel olímpico, Japón es la nación más laureada en judo, pero otros países como Francia, Rusia, Corea del Sur, Brasil, Cuba, Georgia y Mongolia también han alcanzado niveles de élite en este deporte. En total, el judo se practica en más de 200 países.
--------------------------------------------------
I